In [101]:
# This is the setup and 
import re
import subprocess

from astropy.io import fits
import pyfits

# Allow cell output in markdown using printmd function.
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
printmd('**bold**')

In [100]:
ls


29-06-2014UT-ARCSAT/  dataSet.py  LICENSE         masterlog.log
Basic-Notebook.ipynb  images.py   log_handler.py  README.md


In [954]:
def read_obs_log(logName):
    # The Latin-1 encoding is needed because of the degree symobl.
    log = open(logName,'r', encoding='Latin-1')
    log = log.read()
    return log

def extract_log_metadata(log, filesMeta):
    """
    Parse log string and return a list of filenames and a dictionary.
    
    Arguments:
    - ARCSAT log in the form of a sngle string.
    
    Returns:
    - List of filenames(strings) that apear in the log. The file names
      have no .fits extension.
    - Dictionary whith the filenames as keys. The value for each filename
      (key) is a dictionary with the time. 
      
    Parses an arcsat log in the form of a single string; meant to handle
    a string comprised of all the logs for a given night. Finds all image
    file names referanced in the log and creates a list of those names (these
    file names don't have .fits extensions). Builds a dictionary with keys for
    each of these files. The value for each is itself a dictions with keys and
    vlues coresponding to information in the log. Find the time an image was
    finished and weither or not the image was plate solved. If plate solved
    the average FWHM is found.
    """
    
    filenames = []
    # Find the files that have information in the logs.
    searches = re.findall(r'(Imaging to) (\S+)', log)
    [filenames.append(x[1].strip('\n')) for x in searches[:]]
    filenames = [x + '.fits' for x in filenames]
    # Find all the relevant information on each file in the logs.
    for name in filenames:
        nameInLog = name.replace('.fits','')
        # Build a string containing only information for a single file.
        searchstring = r''+nameInLog+'.*?turning tracking off for safety\)'
        singleImageMeta = str(re.findall(searchstring, log, re.DOTALL)[0])
        filesMeta[name] = {}
        # Find the time that a given file was finished.
        finishTime = re.search(r'(\(exposure complete and image downloaded\))(.*?)(\S{8})',
                               singleImageMeta, re.DOTALL)
        finishTime = finishTime.group(3)
        filesMeta[name]['finishTime'] = finishTime
        # Dtermine if an image was plate solved and if so, get the average FWHM.
        if 'Solved!' in singleImageMeta:
            filesMeta[name]['solved'] = True
            fwhm = re.search(r'(Image) (FWHM is) (\d\D\d)', singleImageMeta)
            fwhm = float(fwhm.group(3))
            filesMeta[name]['fwhm'] = fwhm
        else:
            filesMeta[name]['solved'] = False
    return filesMeta

def create_files_meta():
    filesMeta = {}
    # List all fits files in the current dir into a string.
    ls = subprocess.check_output(['ls -1 *.fits'],
                                 universal_newlines=True, shell=True).strip().split('\n')
    # Determine the type of the image by file name, because french fries.
    for i in ls:
        filesMeta[i] = {}
        if 'Dark' in i:
            filesMeta[i]['imtype'] = 'dark'
            print(i)
        elif 'Bias' in i:
            filesMeta[i]['imtype'] = 'bias'
        elif 'skyflat' in i:
            filesMeta[i]['imtype'] = 'skyflat'
        elif 'domeflat' in i:
            filesMeta[i]['imtype'] = 'domeflat'
        else:
            filesMeta[i]['imtype'] = 'object'
            
        
    return filesMeta, ls

In [875]:
cd /home/brandon/Work/ADRL/

/home/brandon/Work/ADRL


In [876]:
log = read_arcsat_obs_log('masterlog.log')

In [877]:
cd 29-06-2014UT-ARCSAT/

/home/brandon/Work/ADRL/29-06-2014UT-ARCSAT


In [957]:
filesMeta, ls = create_files_meta()
filesMeta = extract_log_metadata(log, filesMeta)
print(filesMeta)

Dark_B2_20140629_120359.fits
Dark_B2_20140629_120729.fits
Dark_B2_20140629_121100.fits
{'domeflat_johnson-cousins_V_003.fits': {'imtype': 'domeflat'}, 'skyflat_johnson-cousins_Rc_004.fits': {'imtype': 'skyflat'}, 'NAPG-A-I_johnson-cousins_Ic_20140629_090200.fits': {'fwhm': 2.5, 'solved': True, 'finishTime': '09:05:42'}, 'domeflat_johnson-cousins_V_006.fits': {'imtype': 'domeflat'}, 'skyflat_halpha_6563_50_003.fits': {'imtype': 'skyflat'}, 'NAPG-A-I_johnson-cousins_Ic_20140629_085010.fits': {'fwhm': 2.6, 'solved': True, 'finishTime': '08:53:52'}, 'Bias_B2_20140629_115958.fits': {'imtype': 'bias'}, 'domeflat_johnson-cousins_Ic_002.fits': {'imtype': 'domeflat'}, 'skyflat_johnson-cousins_V_004.fits': {'imtype': 'skyflat'}, 'NAPG-L-V_johnson-cousins_V_20140629_094549.fits': {'fwhm': 2.7, 'solved': True, 'finishTime': '09:49:21'}, 'Bias_B2_20140629_120040.fits': {'imtype': 'bias'}, 'domeflat_johnson-cousins_Rc_003.fits': {'imtype': 'domeflat'}, 'domeflat_johnson-cousins_V_008.fits': {'imtype